In [1]:
from theano.sandbox import cuda
# cuda.use('gpu0')

In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [3]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [4]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600901


In [5]:
path = get_file('lorem-legal-short.txt', origin="https://gist.githubusercontent.com/bearpelican/526781bc838daa2d0babf5b3d6024028/raw/7147fdc558d99a09ea49ff441f872030e8bb4315/lorem-legal-short.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 1579663


In [6]:
!tail {path} -n5

THIS DISCLAIMER OF LIABILITY SHALL NOT BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, OR CONSEQUENTIAL DAMAGES, SO THIS LIMITATION OF LIABILITY EXCEPT AS EXPRESSLY SET FORTH IN THIS AGREEMENT, NEITHER RECIPIENT NOR ANY CONTRIBUTORS SHALL HAVE BEEN INFORMED OF THE POSSIBILITY OF SUCH DAMAGES. GENERAL If any Recipient extracts Embedded Fonts from a Digital Document File in which case the failure of the Licensed Program, or any third party. Description of Modifications. You must duplicate the notice described in Section 4(a) below, each person or entity receiving or distributing Covered Code or executable form under its own expense.

For example, a Distributor might include the Contribution. COMMERCIAL DISTRIBUTION Commercial distributors of software distributed under this License, without any notice. In the absence of any change.

Disclaimer THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS' AND WITHOUT ANY EXPRESS OR IMPLIED INCLUDING, WITHOUT LIMITATION, ANY W

In [7]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 62


In [9]:
chars.insert(0, "\0")

In [10]:
''.join(chars[1:-6])

'\n "$%&\'()*,-./0123456789:;<>?@`abcdefghijklmnopqrstuvwx'

In [11]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
idx = [char_indices[c] for c in text]

In [13]:
idx[:10]

[56, 46, 52, 2, 32, 38, 49, 36, 36, 2]

In [14]:
''.join(indices_char[i] for i in idx[:70])

'you agree to indemnify the licensor except as expressly stated in this'

## Preprocess and create model

In [15]:
maxlen = 40
n_fac = 24
path = "data/legal-short/"
model_path = path + 'models/'

In [16]:
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

KeyboardInterrupt: 

In [ ]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [ ]:
sentences.shape, next_chars.shape

In [ ]:
save_array(model_path + 'sentences.bc', sentences)
save_array(model_path + 'next_chars.bc', next_chars)

In [17]:
sentences = load_array(model_path + 'sentences.bc')
next_chars = load_array(model_path + 'next_chars.bc')

In [18]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [22]:
def print_example(input_string="ethics is a basic foundation of all that"):
    filtered = [x for x in input_string.lower() if x in chars]
    seed_string = ''.join(filtered)[:maxlen]
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [ ]:
print_example("WHEREAS, the board of directors of Parent has unanimously (i) determined that this Agreement")

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

In [ ]:
model.save_weights(model_path+'char_rnn_1.h5')

In [ ]:
print_example("This AGREEMENT AND PLAN OF MERGER this Agreement dated as of February 21 2017")

In [ ]:
sentences[:1000, :].shape

In [ ]:
model.fit(sentences[:500000,:], np.expand_dims(next_chars[:500000,:],-1), batch_size=64, nb_epoch=1)

In [ ]:
model.save_weights(model_path+'char_rnn_2.h5')

In [ ]:
print_example("This AGREEMENT AND PLAN OF MERGER this Agreement dated as of February 21 2017")

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit(sentences[-200000:,:], np.expand_dims(next_chars[-200000:,:],-1), batch_size=64, nb_epoch=1)

In [ ]:
model.save_weights(model_path+'char_rnn_3.h5')

In [ ]:
model.load_weights(model_path+'char_rnn_3.h5')

In [ ]:
print_example("This AGREEMENT AND PLAN OF MERGER this Agreement dated as of February 21 2017")

In [ ]:
model.optimizer.lr=0.0001

In [ ]:
model.fit(sentences[200000:400000,:], np.expand_dims(next_chars[200000:400000,:],-1), batch_size=64, nb_epoch=1)

In [ ]:
print_example("This AGREEMENT AND PLAN OF MERGER this Agreement dated as of February 21 2017")

In [ ]:
model.save_weights(model_path+'char_rnn_4.h5')

In [ ]:
model.load_weights(model_path+'char_rnn_4.h5')

In [ ]:
model.optimizer.lr=0.0001

In [ ]:
model.fit(sentences[400000:900000,:], np.expand_dims(next_chars[400000:900000:,:],-1), batch_size=64, nb_epoch=1)

In [ ]:
model.save_weights(model_path+'char_rnn_5.h5')

In [20]:
model.load_weights(model_path+'char_rnn_5.h5')

In [24]:
print_example("Booger brains are delicious but zombies don't like to eat them")

booger brains are delicious but zombies contributors, as long as a whole, and you must make it clear that you may subject to the authors in the work is provided by the contributor to version of the licensed program contain terms. you may modify covered code. however caused and on any out of the covered code or modifications as derived from the instructions d


In [ ]:
model.optimizer.lr=0.00001

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

In [ ]:
print_example()

In [ ]:
model.save_weights(model_path+'char_rnn_6.h5')

In [ ]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

In [ ]:
print_example()

In [ ]:
print_example()

In [ ]:
model.save_weights(model_path+'char_rnn_6.h5')

### Train on real legal document

In [ ]:
path = get_file('legal-pop.txt', origin="https://gist.githubusercontent.com/bearpelican/3b2aad7b23266651ec70455561a1f2a6/raw/92c604883da6fa59a4abeb646c2b069e121f8746/popeyes_legal.txt")
rl_text = open(path).read().lower()
print('corpus length:', len(rl_text))

In [ ]:
!tail {path} -n5

In [ ]:
rl_text = [x for x in rl_text if x in chars]

In [ ]:
rl_idx = [char_indices[c] for c in rl_text]

In [ ]:
rl_idx[:10]

In [ ]:
''.join(indices_char[i] for i in rl_idx[:70])

## Preprocess and create model

In [ ]:
rl_sentences = []
rl_next_chars = []
for i in range(0, len(rl_idx) - maxlen+1):
    rl_sentences.append(rl_idx[i: i + maxlen])
    rl_next_chars.append(rl_idx[i+1: i+maxlen+1])
print('nb sequences:', len(rl_sentences))

In [ ]:
rl_sentences = np.concatenate([[np.array(o)] for o in rl_sentences[:-2]])
rl_next_chars = np.concatenate([[np.array(o)] for o in rl_next_chars[:-2]])

In [ ]:
rl_sentences.shape, rl_next_chars.shape

In [ ]:
path = "data/legal-short/"
model_path = path + 'models/'
prefix = 'legal_pop'
save_array(model_path + prefix + 'sentences_pop.bc', sentences)
save_array(model_path + prefix + 'next_chars_pop.bc', next_chars)

In [ ]:
model.load_weights(model_path+'char_rnn_5.h5')

In [ ]:
model.optimizer.lr=0.01

In [ ]:
model.fit(rl_sentences, np.expand_dims(rl_next_chars,-1), batch_size=64, nb_epoch=2)

In [ ]:
model.save_weights(model_path+'char_rnn_pop_1.h5')

In [ ]:
print_example("Here is a random text generated by this thing")